# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [1]:
# Load the libraries as required.
%load_ext dotenv
%dotenv 
import os
import sys
# sys.path.append(os.getenv('SRC_DIR'))
import pandas as pd
import numpy as np

cannot find .env file


In [2]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [3]:
fires_dt

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00


# Get X and Y

Create the features data frame and target data.

In [4]:
fires_dt.columns

Index(['coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi',
       'temp', 'rh', 'wind', 'rain', 'area'],
      dtype='object')

In [5]:
X = fires_dt.drop('area', axis=1)
X

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0


In [6]:
Y = fires_dt['area']
Y

0       0.00
1       0.00
2       0.00
3       0.00
4       0.00
       ...  
512     6.44
513    54.29
514    11.16
515     0.00
516     0.00
Name: area, Length: 517, dtype: float64

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

numerical_vars = [
                    # 'coord_x', 'coord_y', 
                  'ffmc', 'dmc', 'dc', 'isi',
                    'temp', 'rh', 'wind', 'rain'
                    # ,'area'
                    ]  # numerical columns
categorical_vars = X.select_dtypes(include=['object']).columns   # categorical columns

In [30]:
# Scaling numerical features using StandardScaler and OneHotEncoder for categorical features
preproc1 = ColumnTransformer([
    # Handle numerical columns: scaling with StandardScaler
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),  # Handle missing values
        ('scaler', StandardScaler())
    ]), numerical_vars),
    
    # Handle categorical columns: one-hot encoding
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Handle missing categorical values
        # ('onehot', OneHotEncoder())  # One-hot encoding
        ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encoding
    ]), categorical_vars)
])
preproc1

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh',
                                  'wind', 'rain']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 Index(['month', 'day'], dtype='object'))])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [31]:
preproc2 = ColumnTransformer([
    # Handle numerical columns: scaling with PowerTransformer (non-linear)
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),  # Handle missing values
        ('scaler', StandardScaler())
    ]), numerical_vars),
    
    # Handle categorical columns: one-hot encoding
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  # Handle missing categorical values
        # ('onehot', OneHotEncoder())  # One-hot encoding
        ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encoding
    ]), categorical_vars)
])
preproc2


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh',
                                  'wind', 'rain']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 Index(['month', 'day'], dtype='object'))])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [32]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import r2_score, mean_squared_error
from xgboost import XGBRegressor

In [33]:
# Pipeline A = preproc1 + baseline
Pipeline_A = Pipeline([
    ('preprocessing', preproc1),  
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])
Pipeline_A

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['ffmc', 'dmc', 'dc', 'isi',
                                                   'temp', 'rh', 'wind',
                                                   'rain']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['month', 'day'], dtype='object'))])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [34]:
# Pipeline B = preproc2 + baseline
Pipeline_B = Pipeline([
    ('preprocessing', preproc2),  
    ('regressor', GradientBoostingRegressor(random_state=42))
])
Pipeline_B

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['ffmc', 'dmc', 'dc', 'isi',
                                                   'temp', 'rh', 'wind',
                                                   'rain']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['month', 'day'], dtype='object'))])),
                ('regressor', GradientBoostingRegressor(random_state=42))])

In [35]:
# Pipeline C = preproc1 + advanced model
Pipeline_C = Pipeline([
    ('preprocessing', preproc1),  
    ('regressor', LinearRegression())
])
Pipeline_C

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['ffmc', 'dmc', 'dc', 'isi',
                                                   'temp', 'rh', 'wind',
                                                   'rain']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['month', 'day'], dtype='object'))])),
                ('regressor', LinearRegression())])

In [36]:
# Pipeline D = preproc2 + advanced model
Pipeline_D = Pipeline([
    ('preprocessing', preproc2),  
    ('regressor', XGBRegressor(random_state=42, objective='reg:squarederror'))
])
Pipeline_D
    

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['ffmc', 'dmc', 'dc', 'isi',
                                                   'temp', 'rh', 'wind',
                                                   'rain']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [37]:
# Grid for Baseline Model with Preproc1 
Pipeline_A_param_grid = {
    'regressor__n_estimators': [50, 100, 200, 300], 
    'regressor__max_depth': [5, 10, 15, None],  
    'regressor__min_samples_split': [2, 5, 10, 20],  
    'regressor__min_samples_leaf': [1, 2, 4, 6]  
}

# Grid for Baseline Model with Preproc2
Pipeline_B_param_grid = {
    'regressor__n_estimators': [50, 100, 200, 300],  
    'regressor__learning_rate': [0.01, 0.05, 0.1, 0.2],  # Step size
    'regressor__max_depth': [3, 5, 7, 10],  
    'regressor__subsample': [0.8, 0.9, 1.0, 1.0]  
}

# Grid for Advance Model  with Preproc1
Pipeline_C_param_grid = {
    # 'regressor__fit_intercept': [True, False]
    'regressor__copy_X': [True,False], 
    'regressor__fit_intercept': [True,False], 
    'regressor__n_jobs': [1,5,10,15,None], 
    'regressor__positive': [True,False]
}

# Grid for Advance Model with Preproc2
Pipeline_D_param_grid = {
    'regressor__n_estimators': [50, 100, 200, 300],  # Number of boosting stages
    'regressor__learning_rate': [0.01, 0.05, 0.1, 0.2],  # Step size
    'regressor__max_depth': [3, 5, 7, 10], 
    'regressor__subsample': [0.8, 0.9, 1.0, 1.0] 
}

In [38]:
# RandomForestRegressor Preproc1
grid_search_lr_A = GridSearchCV(
                    Pipeline_A, 
                    Pipeline_A_param_grid, 
                    cv=5, 
                    scoring='neg_mean_squared_error', 
                    n_jobs=-1
                    )
grid_search_lr_A
grid_search_lr_A.fit(X_train, y_train)

print(f"Best parameters for RandomForestRegressor pipeline A: {grid_search_lr_A.best_params_}")
best_lr_A_model = grid_search_lr_A.best_estimator_

# Evaluate performance
y_pred_lr_A = best_lr_A_model.predict(X_test)
mse_lr_A = mean_squared_error(y_test, y_pred_lr_A)
r2_lr_A = r2_score(y_test, y_pred_lr_A)
print(f"RandomForestRegressor - MSE: {mse_lr_A}, R2: {r2_lr_A}")

Best parameters for RandomForestRegressor pipeline A: {'regressor__max_depth': 5, 'regressor__min_samples_leaf': 6, 'regressor__min_samples_split': 20, 'regressor__n_estimators': 300}
RandomForestRegressor - MSE: 11646.72395713549, R2: 0.011964795533731865


In [39]:
# GradientBoostingRegressor Preproc1
grid_search_lr_B = GridSearchCV(
                    Pipeline_B, 
                    Pipeline_B_param_grid, 
                    cv=5, 
                    scoring='neg_mean_squared_error', 
                    n_jobs=-1
                    )
grid_search_lr_B
grid_search_lr_B.fit(X_train, y_train)

print(f"Best parameters for GradientBoostingRegressor pipeline B: {grid_search_lr_B.best_params_}")
best_lr_B_model = grid_search_lr_B.best_estimator_

# Evaluate performance
y_pred_lr_B = best_lr_B_model.predict(X_test)
mse_lr_B = mean_squared_error(y_test, y_pred_lr_B)
r2_lr_B = r2_score(y_test, y_pred_lr_B)
print(f"GradientBoostingRegressor - MSE: {mse_lr_B}, R2: {r2_lr_B}")

Best parameters for GradientBoostingRegressor pipeline B: {'regressor__learning_rate': 0.01, 'regressor__max_depth': 3, 'regressor__n_estimators': 50, 'regressor__subsample': 0.8}
GradientBoostingRegressor - MSE: 11768.72223331677, R2: 0.0016152249424709053


In [40]:
# LinearRegression Preproc1
grid_search_rfr_C = GridSearchCV(
                    Pipeline_C, 
                    Pipeline_C_param_grid, 
                    cv=5, 
                    scoring='neg_mean_squared_error', 
                    n_jobs=-1
                    )
grid_search_rfr_C
grid_search_rfr_C.fit(X_train, y_train)

print(f"Best parameters for LinearRegression pipeline_C: {grid_search_rfr_C.best_params_}")
best_rfr_C_model = grid_search_rfr_C.best_estimator_

# Evaluate performance
y_pred_rfr_C = best_rfr_C_model.predict(X_test)
mse_rfr_C = mean_squared_error(y_test, y_pred_rfr_C)
r2_rfr_C = r2_score(y_test, y_pred_rfr_C)
print(f"LinearRegression pipeline_C - MSE: {mse_rfr_C}, R2: {r2_rfr_C}")

Best parameters for LinearRegression pipeline_C: {'regressor__copy_X': True, 'regressor__fit_intercept': False, 'regressor__n_jobs': 1, 'regressor__positive': True}
LinearRegression pipeline_C - MSE: 11750.500426783201, R2: 0.003161049022306517


In [41]:
# XGBoostRegressor Preproc1
grid_search_rfr_D = GridSearchCV(
                    Pipeline_D, 
                    Pipeline_D_param_grid, 
                    cv=5, 
                    scoring='neg_mean_squared_error', 
                    n_jobs=-1
                    )
grid_search_rfr_D
grid_search_rfr_D.fit(X_train, y_train)

print(f"Best parameters for XGBoostRegressor pipeline_D: {grid_search_rfr_D.best_params_}")
best_rfr_D_model = grid_search_rfr_D.best_estimator_

# Evaluate performance
y_pred_rfr_D = best_rfr_D_model.predict(X_test)
mse_rfr_D = mean_squared_error(y_test, y_pred_rfr_D)
r2_rfr_D = r2_score(y_test, y_pred_rfr_D)
print(f"XGBoostRegressor pipeline_D- MSE: {mse_rfr_D}, R2: {r2_rfr_D}")

Best parameters for XGBoostRegressor pipeline_D: {'regressor__learning_rate': 0.01, 'regressor__max_depth': 3, 'regressor__n_estimators': 50, 'regressor__subsample': 0.8}
XGBoostRegressor pipeline_D- MSE: 11770.911381734519, R2: 0.0014295112849223024


# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

In [21]:
#RandomForestRegressor Pipeline A has the best performance. 

import pickle
# best model 
best_model = best_rfr_C_model

# Define the file path where the model will be saved
model_filename = '../../05_src/data/fires/best_forest_fire_model.pkl'

# Save the model to the file
with open(model_filename, 'wb') as file:
    pickle.dump(best_model, file)

print(f"Best model saved to {model_filename}")

Best model saved to ../../05_src/data/fires/best_forest_fire_model.pkl


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [28]:
import shap
from shap import TreeExplainer
from sklearn.ensemble import RandomForestClassifier

# Create a SHAP explainer for the XGBoost model
explainer = shap.TreeExplainer(best_rfr_D_model.named_steps['regressor'])

# Select an observation from the test set
observation_idx = 42

# Get the test set observation's features (X_test is assumed to be already preprocessed)
observation = X_train.iloc[observation_idx:observation_idx+1]
observation
# # Get SHAP values for the observation
# shap_values = explainer.shap_values(observation)


,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain
416,6,3,jul,tue,92.7,164.1,575.8,8.9,26.3,39,3.1,0.0


In [ ]:
# features with higher observation value is important feature and features with lower observation value is least important feature. 
# By checking the value looks like 'rain' and 'wind' are least important feature and 'dc' and 'dmc; are most important feature. 

*(Answer here.)*

## Criteria

The [rubric](./assignment_3_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.